In [1]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
output_path = 'D:/llda/lda/result'
file_path = 'D:/llda/lda/data'
os.chdir(file_path)
data=pd.read_excel("2022.xlsx")#content type
os.chdir(output_path)
stop_file = "D:/llda/lda/stop_dic/stopwords.txt"

In [3]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def english_word_cut(mytext):
    stop_list = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    word_list = []
    tokens = word_tokenize(mytext)
    
    for word in tokens:
        word = re.sub(r'[^A-Za-z]', '', word)
        word = word.lower()
        
        if word not in stop_list and len(word) >= 2:
            word = lemmatizer.lemmatize(word)
            word_list.append(word)
    
    return " ".join(word_list)

In [4]:
data["content_cutted"] = data.content.apply(english_word_cut)

In [5]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data.content_cutted)
feature_names = vectorizer.get_feature_names_out()

keywords_list = []
for doc_index, doc in enumerate(data.content_cutted):
    feature_index = tfidf_matrix[doc_index, :].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc_index, x] for x in feature_index])
    keywords = [(feature_names[i], score) for (i, score) in tfidf_scores]
    keywords = sorted(keywords, key=lambda x: x[1], reverse=True)
    
    #print(f"Document {doc_index + 1} keywords:")
    #for keyword, score in keywords:
        #print(f"{keyword}: {score}")
    #print()
    keywords_list.append(keywords)

df = pd.DataFrame(keywords_list)
df.to_excel("keywords2022_output.xlsx", index=False)